In [1]:
import ROOT
from ipywidgets import interact, IntSlider, IntText
from itertools import takewhile, count

f = ROOT.TFile('outWithRiderFits.root')
module_label = 'Calo23RiderFits'

if not ROOT.gROOT.FindObject('canv'):
    c = ROOT.TCanvas('canv', 'canv', 1000, 600)
sgen = takewhile(lambda s: isinstance(s, ROOT.TSpline3), 
                         (f.Get('{0}/splines/spline{1}'.format(module_label, i)) 
                          for i in count()))
splines = list(sgen)
t = f.Get('{}/fits/fitTree'.format(module_label))
n_entries = t.BuildIndex('fillNum', 'xtalNum*(waveformIndex == 1) - 1*(waveformIndex != 1)')

# need global handle to currently plotted objects so that they don't get garbage collected
plotted_objects = None

Welcome to JupyROOT 6.08/02


In [2]:
# define plotting functions

def plot_fit(entry_num, x_min, x_max, canvas):
    t.GetEntry(entry_num)
 
    # build graph
    sample_g = ROOT.TGraph(0)
    sample_g.SetTitle('run {0}, fill {2}, waveform {3}, xtal {1}; sample num; adc counts'.
                          format(t.runNum, t.xtalNum, t.fillNum, t.waveformIndex))
    sample_g.SetMarkerStyle(20)
    sample_g.SetMarkerSize(1.5)
    sample_g.SetMarkerColor(ROOT.kBlack)
    for i, samp in enumerate(t.trace):
        sample_g.SetPoint(i, i + t.firstSampleNum, samp)
    
    # build fit function
    try:
        spline = splines[t.xtalNum]
    except IndexError:
        print('no spline found for segment {}!'.format(t.xtalNum))
        return
    
    # need to copy parameters or closure doesn't work as intended
    energies = [e for e in t.energies]
    times = [float(t0) for t0 in t.times]
    ped = float(t.pedestal)
    def fit_func(x):
        val = ped
        for (energy, t0) in zip(energies, times):
            if spline.GetXmin() < (x[0]-t0) < spline.GetXmax():
                val += -1*energy*spline.Eval(x[0]-t0)
        return val
    
    # draw graph and fit 
    sample_g.Draw("ap")
    y_min, y_max = (sample_g.GetYaxis().GetXmin(), sample_g.GetYaxis().GetXmax())
    sample_g.GetXaxis().SetLimits(x_min, x_max)
    fitTF1 = ROOT.TF1('fitfunc', fit_func, x_min, x_max, 0)
    fitTF1.SetNpx(1000)
    fitTF1.SetLineColor(ROOT.kMagenta + 2)
    # don't draw failed fits
    if t.chi2 > 0:
        fitTF1.Draw("p same")
    
    # make text box
    x_range, y_range = (x_max - x_min, y_max - y_min)
    txtbox = ROOT.TPaveText(x_min + 0.5*x_range, y_min + 0.6*y_range, 
                            x_min + 0.9*x_range, y_min + 0.1*y_range)
    txtbox.SetFillColor(ROOT.kWhite)
    txtbox.SetLineColor(ROOT.kWhite)
    txtbox.SetShadowColor(ROOT.kWhite)
    for (i, (energy, t0)) in enumerate(zip(t.energies, t.times)):
        txtbox.AddText('t_{{{0}}}: {1:0.3f}'.format(i, t0))
        txtbox.AddText('E_{{{0}}}: {1:0.1f}'.format(i, energy))
    txtbox.AddText('pedestal: {:0.1f}'.format(t.pedestal))
    txtbox.AddText('#chi^{{2}}: {:0.1f}'.format(t.chi2))
    sample_g.GetYaxis().SetTitleOffset(1.5)
    txtbox.Draw()
    canvas.Draw()
    return (sample_g, fitTF1, txtbox)

def plot_one_with_index(fill_num, xtal_num, waveform, x_min, x_max):
    ROOT.gStyle.SetOptTitle(1)
    entry_num = t.GetEntryNumberWithIndex(fill_num, xtal_num)
    if entry_num != -1L:
        c.cd()
        global plotted_objects
        plotted_objects = plot_fit(entry_num + waveform - 1, x_min, x_max, c)
    else:
        print('entry not found in index')

In [3]:
%jsroot on
dummy = interact(plot_one_with_index, fill_num=IntSlider(min=1, max=100, step=1, value=0), 
                 waveform=IntSlider(min=1, max=5, step=1, value=0), 
                 xtal_num=IntSlider(min=0, max=len(splines) - 1, step=1, value=0),
                 x_min=IntSlider(min=0, max=800, step=1, value=0),
                 x_max=IntSlider(min=0, max=800, step=1, value=800))